In [2]:
#from google.colab import drive
#drive.mount('/content/drive',force_remount=True)

In [3]:
dataset_path = "/Users/sid/Downloads/Food Classification dataset"

In [4]:
from tensorflow import keras
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
BATCH_SIZE = 64

datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.25
)
train_data = datagen.flow_from_directory(
    directory=dataset_path,
    target_size=(299,299),
    batch_size=BATCH_SIZE,
    subset='training',
    class_mode = 'categorical',
    seed=64
)
valid_data = datagen.flow_from_directory(
    directory=dataset_path,
    target_size=(299,299),
    batch_size=BATCH_SIZE,
    subset='validation',
    class_mode = 'categorical',
    seed=64
)

Found 9066 images belonging to 28 classes.
Found 3009 images belonging to 28 classes.


In [6]:
base_model = InceptionResNetV2(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(299, 299, 3),
    include_top=False,
)

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(299, 299, 3))

# scaled from (0, 255) to a range of (-1., +1.)
scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(inputs)

# base_model is running in inference mode here.
x = base_model(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(28, activation = 'softmax')(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer = keras.optimizers.Adam(learning_rate = 0.001) , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 299, 299, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 299, 299, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_resnet_v2             │ (None, 8, 8, 1536)     │    54,336,736 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1536)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1536)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 28)             │        43,036 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 54,379,772 (207.44 MB)

 Trainable params: 43,036 (168.11 KB)

 Non-trainable params: 54,336,736 (207.28 MB)

In [5]:
base_model.trainable = True
model.compile(optimizer = keras.optimizers.Adam(learning_rate = 0.001),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

epochs = 5
incResNet = model.fit(
    train_data,
    epochs=epochs,
    validation_data=valid_data,
)

Epoch 1/5


/Users/sid/anaconda3/miniconda3/envs/myenv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 75/142 ━━━━━━━━━━━━━━━━━━━━ 26:56 24s/step - accuracy: 0.5154 - loss: 1.6861

/Users/sid/anaconda3/miniconda3/envs/myenv/lib/python3.10/site-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


142/142 ━━━━━━━━━━━━━━━━━━━━ 3823s 27s/step - accuracy: 0.5911 - loss: 1.4053 - val_accuracy: 0.3347 - val_loss: 2.5661
Epoch 2/5
142/142 ━━━━━━━━━━━━━━━━━━━━ 3584s 25s/step - accuracy: 0.8418 - loss: 0.4966 - val_accuracy: 0.6520 - val_loss: 1.4287
Epoch 3/5
142/142 ━━━━━━━━━━━━━━━━━━━━ 3627s 26s/step - accuracy: 0.8942 - loss: 0.3173 - val_accuracy: 0.6314 - val_loss: 1.5842
Epoch 4/5
142/142 ━━━━━━━━━━━━━━━━━━━━ 3812s 27s/step - accuracy: 0.9151 - loss: 0.2637 - val_accuracy: 0.7587 - val_loss: 0.8108
Epoch 5/5
142/142 ━━━━━━━━━━━━━━━━━━━━ 3500s 25s/step - accuracy: 0.9331 - loss: 0.2114 - val_accuracy: 0.6520 - val_loss: 1.5369
